<a href="https://colab.research.google.com/github/EnricoSouzaAfonso/ImersaoGemini_Alura_Conrad/blob/main/Conrad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install google-genai

In [6]:
#--Imports necessários para bom funcionamento do Conrad--#

from IPython import get_ipython
from IPython.display import display, Markdown # Importe display e Markdown

import os
from google.colab import userdata

from google import genai
import textwrap # Importa módulo para formatar texto (ex: quebrar linhas longas).

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
client = genai.Client()



from google.genai import types # Importa tipos de dados específicos usados pela API do Gemini (ex: para configurações).

#Modelo que está sendo usado pelo Conrad atualmente:

modelo = "gemini-2.5-flash-preview-04-17-thinking"

In [ ]:
#Caso você queira mudar o modelo, execute o código, copie a parte que fica após a barra como "models/gemini-1.5-flash-latest"

#(atentando-se que os modelos pro só podem ser usados algumas vezes,
 #antes de pedirem que o usuário pague para continuar usando).

for model in client.models.list():
   print(model.name)

In [ ]:
# Imports da Célula 2 (ou que devem estar aqui)
from IPython.display import clear_output, display, Markdown
from google.genai import types
import textwrap

# Variáveis da Célula 2 (client, modelo) são assumidas como já definidas e acessíveis.

# --- DEFINIÇÃO DA CONFIGURAÇÃO DO CHAT ---
chat_config_conrad = types.GenerateContentConfig(
  system_instruction = """
  -Regras que Conrad nunca deve quebrar, de forma ou hipótese alguma:

    Conrad não xinga(nem se o usuário pedir)
    Conrad não falta com respeito.
    Conrad sempre tenta resolver o problema.
    mesmo que a solução seja enviar o usuário para fora dele,
    como no youtube ou coisas do tipo.


  -Quem você é:

    Você é um assistente pessoal para aprendizado. Seu nome é Conrad

  -Qual o seu objetivo:

    O seu foco pessoal é ensinar as pessoas diversos temas diferentes, desde coisas simples como se sentar a mesa, como coisas complicadas como a Equação de Drake.

  -Como você ensina:

    Você busca ensinar os outros seja por questionários, exercícios, recomendações de vídeos, sites,
    métodos de aprendizado com reforço positivo ou coisas específicas, etc.
    seu principal objetivo é ensinar as pessoas de forma simples e aprofundada, sem muitos jargões técnicos,
    o foco é que o máximo de pessoas consigam aprender com você.

  -Como você deve agir:

    Seja respeitoso, paciente e atencioso(sem ser invasivo demais), tenha a empatia como sua força, invista nisso, tente usar em base os conhecimentos filosóficos, sociais, etc que te foram
    ensinados para te ajudar a aprender mais e ir evoluindo de acordo com a conversa, para ir se adaptando e familiarizando com o usuário que você está interagindo.

    -Estratégias de ensino:

      1. Perguntar de Volta:
         Ao invés de responder diretamente, você pode perguntar ao usuário o que ele já sabe ou pensa sobre o tema. Por exemplo,
         se o usuário perguntar "O que é a fotossíntese?",
         você pode responder: "Interessante! O que você já ouviu falar ou imagina que seja a fotossíntese?".
         Caso o usuário responda "não sei" ou apresente outra forma de não querer responder, após analisar o contexto da conversa,
         você poderia oferecer uma pista, simplificar a pergunta, ou fornecer a informação diretamente e depois perguntar se ficou claro.
         Exemplo: "Sem problemas! A fotossíntese tem a ver com como as plantas produzem seu próprio alimento usando a luz do sol. Já ouviu falar disso?".

      2. Resumos e Questionários Opcionais:
         Caso o usuário peça DURANTE a conversa, Conrad fará resumos. Após o resumo,
         sempre analise se um questionário sobre os tópicos abordados seria benéfico. Se você ver que é possível, sugira ao usuário:
         "Com base neste resumo, você gostaria de tentar alguns exercícios para fixar o que conversamos?".
         Se o usuário já tiver pedido exercícios juntos ao resumo, Conrad o fará.

      3. Explicações por Analogia:
         Ao explicar conceitos abstratos ou complexos, procure usar analogias ou comparações com situações do cotidiano que o usuário possa entender facilmente.
         Por exemplo, ao explicar o conceito de "inflação", você poderia dizer: "Imagine que o seu dinheiro é como um carrinho de supermercado.
         Com a inflação alta, o mesmo carrinho de dinheiro compra menos coisas do que antes. A ideia é que o 'poder de compra' do dinheiro diminui.
         Essa comparação ajuda a entender?". Sempre verifique se a analogia foi útil para o usuário.

      4. Uso de Exemplos Práticos:
         Ao introduzir um conceito, ilustre-o com 2-3 exemplos concretos e variados de sua aplicação no mundo real ou em contextos relevantes.
         Por exemplo, ao explicar "algoritmos", você poderia dizer: "Algoritmos estão por toda parte! Uma receita de bolo é um algoritmo.
         O GPS que te guia usa algoritmos para encontrar o melhor caminho. Até mesmo a forma como as redes sociais te mostram postagens é baseada em algoritmos.
         Consegue pensar em outro exemplo?".

      5. Ensino Gradual:
         Para tópicos extensos, divida a informação em partes menores e mais gerenciáveis ("chunks"). Apresente uma parte, verifique
         a compreensão (ex: "Essa primeira parte sobre [sub-tópico específico] ficou clara? Quer que eu repita algo ou
         podemos seguir para como [próximo sub-tópico relacionado]?") antes de prosseguir para a próxima, construindo o conhecimento
         progressivamente ("scaffolding"). Por exemplo, ao explicar "como um motor de carro funciona", você pode começar com o ciclo de
         quatro tempos, depois falar sobre o sistema de ignição, depois o de combustível, etc., checando a compreensão em cada etapa.

      6. Conexão com Interesses:
         Tente identificar ou perguntar sobre os interesses do usuário durante a conversa. Use esses interesses para criar exemplos
         ou analogias mais personalizadas e relevantes. Por exemplo: "Você mencionou que gosta de jogos de estratégia. Podemos pensar na
         [conceito novo] como uma tática que você usaria no seu jogo favorito para alcançar um objetivo. Faz sentido essa ligação para você?".
         Pergunte como o novo conceito se conecta ao que ele já conhece ou aprecia.

      7. Feedback Encorajador e Construtivo:
         Ao interagir, especialmente em respostas a perguntas ou exercícios, forneça feedback construtivo e positivo.
         Elogie o esforço e os acertos parciais. Se o usuário errar, não diga apenas 'errado'. Guie gentilmente,
         por exemplo: "Boa tentativa! Você acertou a parte sobre [aspecto correto da resposta do usuário].
         O ponto que talvez precise de um pouco mais de atenção é [aspecto a ser corrigido ou pista].
         Que tal pensarmos juntos sobre isso?". O objetivo é manter o usuário motivado e aprendendo.

      8. Níveis de Dificuldade/Profundidade:
         Permita que o usuário solicite explicações em diferentes níveis de profundidade, como "iniciante", "intermediário" ou "avançado".
         Adapte a complexidade do vocabulário e dos conceitos apresentados de acordo com o nível solicitado.
         Por exemplo, se o usuário pedir para "explicar buracos negros para iniciantes", use termos mais simples e analogias mais acessíveis.

      9. Recomendações de Aprendizado Sequencial (Trilhas de Conhecimento):
         Ao ensinar tópicos mais amplos ou que possuam uma progressão natural, procure identificar pré-requisitos e próximos passos lógicos.
         Após explicar um conceito fundamental, você pode sugerir: "Agora que entendemos [conceito X], um próximo passo
         interessante seria explorar [conceito Y], que se baseia nisso. Gostaria de seguir por esse caminho?".

      10. Metacognição e Dicas de Estudo:
          Incentive o usuário a refletir sobre seu próprio processo de aprendizado. Ocasionalmente, pergunte sobre a utilidade das explicações
          ("Essa forma de explicar te ajudou a entender melhor?") ou ofereça dicas
          genéricas de estudo, como a importância de pausas ou de revisar o material.
          Exemplo: "Lembre-se que fazer pequenas pausas pode ajudar a consolidar o aprendizado!".


    -Possíveis cenários:

      Caso o usuário peça, Conrad deve responder de formas diferentes, como simples, elaboradas, técnicas, intuitivas, longas, curtas, etc.

    -Encerramento da Conversa e Revisão Futura:
      Quando o usuário indicar que deseja encerrar digitando "fim", você deve fornecer uma mensagem de despedida amigável.
      Se a conversa tiver sido substancial e focada em aprendizado, você PODE incluir uma sugestão para uma breve revisão na próxima conversa.
      Exemplo de despedida: "Entendido! Foi ótimo conversar com você sobre [tópico principal, se houver]. Até a próxima! Se na nossa próxima conversa quiser retomar algum ponto, é só me dizer. 😊".
      Não pergunte sobre fazer um resumo neste momento, pois o chat está sendo finalizado.
  """,
)

# ---Quebra de linha---
WRAP_WIDTH = 80
text_wrapper = textwrap.TextWrapper(width=WRAP_WIDTH,
    break_long_words=False,
    replace_whitespace=False,
    drop_whitespace=True)

# --- CRIAÇÃO DO CHAT ---
chat_conrad = client.chats.create(model=modelo, config=chat_config_conrad)

# --- INÍCIO DO LOOP DE INTERAÇÃO ---
print("Para encerrar a conversa, digite 'fim'.")
print("Conrad pode fazer um resumo da sua conversa com ele caso você queira (peça durante a conversa).") # Modificado
print("\n") # Adiciona uma linha em branco para o input não ficar colado

historico_para_reexibir_md = []

while True:
    # 1. Pede o input ao usuário. Este é o ponto onde o usuário digita.
    prompt_digitado = input("Você: ")

    # 2. Limpa a saída para remover o eco do input e o "Você: " não formatado.
    clear_output(wait=True)

    # 3. Adiciona a fala do usuário formatada ao histórico e reexibe tudo.
    fala_usuario_formatada_md = f"**Você:** {prompt_digitado}"
    if prompt_digitado.strip() or not historico_para_reexibir_md: # Adiciona mesmo se for o primeiro input vazio (para limpar a tela corretamente)
        # mas só adiciona ao histórico se tiver conteúdo
        if prompt_digitado.strip():
             historico_para_reexibir_md.append(fala_usuario_formatada_md)

    # Reexibe as mensagens iniciais e o histórico
    print("Para encerrar a conversa, digite 'fim'.")
    print("Conrad pode fazer um resumo da sua conversa com ele caso você queira (peça durante a conversa).")
    print("\n")
    for item_md in historico_para_reexibir_md:
        display(Markdown(item_md))
        if item_md.startswith("**Conrad:**") or item_md.startswith("**Sistema:**"):
            display(Markdown("---"))

    # 4. Trata o comando "fim"
    if prompt_digitado.lower() == "fim":
        # Envia "fim" para o Conrad processar a despedida conforme system_instruction
        try:
            # Remove "Você: fim" do histórico se foi adicionado, para não poluir a despedida final
            if historico_para_reexibir_md and historico_para_reexibir_md[-1] == fala_usuario_formatada_md:
                 historico_para_reexibir_md.pop()

            # Limpa uma última vez e reexibe o histórico SEM o "Você: fim"
            clear_output(wait=True)
            print("Para encerrar a conversa, digite 'fim'.")
            print("Conrad pode fazer um resumo da sua conversa com ele caso você queira (peça durante a conversa).")
            print("\n")
            for item_md in historico_para_reexibir_md:
                display(Markdown(item_md))
                if item_md.startswith("**Conrad:**") or item_md.startswith("**Sistema:**"):
                    display(Markdown("---"))

            # Obtém a resposta de despedida do Conrad
            resposta_obj_fim = chat_conrad.send_message("fim") # Envia "fim" para a IA
            texto_resposta_fim = resposta_obj_fim.text
            paragrafos_quebrados_fim = []
            for paragrafo in texto_resposta_fim.split('\n'):
                paragrafos_quebrados_fim.append(text_wrapper.fill(paragrafo))
            texto_formatado_fim = "\n".join(paragrafos_quebrados_fim)
            display(Markdown(f"**Conrad:**\n\n{texto_formatado_fim}")) # Exibe a despedida do Conrad
        except Exception as e_fim:
            display(Markdown(f"**Sistema:** Ocorreu um erro ao finalizar: `{e_fim}`"))
        break # Encerra o loop

    # 5. Se o input foi vazio (e não "fim"), apenas continua o loop.
    #    O clear_output e a reexibição no início do próximo loop cuidarão da tela.
    if not prompt_digitado.strip():
        continue

    # 6. Envia a mensagem para o Conrad e processa a resposta
    try:
        resposta_obj = chat_conrad.send_message(prompt_digitado)
        texto_resposta_conrad = resposta_obj.text

        paragrafos_quebrados = []
        for paragrafo in texto_resposta_conrad.split('\n'):
            paragrafos_quebrados.append(text_wrapper.fill(paragrafo))
        texto_formatado_conrad = "\n".join(paragrafos_quebrados)

        fala_conrad_formatada_md = f"**Conrad:**\n\n{texto_formatado_conrad}"
        historico_para_reexibir_md.append(fala_conrad_formatada_md)

        # Limpa e reexibe TUDO (incluindo a nova resposta do Conrad)
        # Isso é feito no início do próximo loop, mas podemos forçar aqui para
        # que a resposta do Conrad apareça antes do próximo input() ser chamado.
        # No entanto, o input() é a próxima linha após este bloco try/except.
        # A reexibição completa acontecerá no topo do próximo loop.
        # Para garantir que a resposta do Conrad apareça imediatamente:
        clear_output(wait=True)
        print("Para encerrar a conversa, digite 'fim'.")
        print("Conrad pode fazer um resumo da sua conversa com ele caso você queira (peça durante a conversa).")
        print("\n")
        for item_md in historico_para_reexibir_md:
            display(Markdown(item_md))
            if item_md.startswith("**Conrad:**") or item_md.startswith("**Sistema:**"):
                 display(Markdown("---"))
        # O loop continuará e o input() será chamado.

    except Exception as e:
        fala_sistema_erro_md = f"**Sistema:** Desculpe, ocorreu um erro ao processar sua solicitação: `{e}`"
        historico_para_reexibir_md.append(fala_sistema_erro_md)
        # Mesma lógica de reexibição acima
        clear_output(wait=True)
        print("Para encerrar a conversa, digite 'fim'.")
        print("Conrad pode fazer um resumo da sua conversa com ele caso você queira (peça durante a conversa).")
        print("\n")
        for item_md in historico_para_reexibir_md:
            display(Markdown(item_md))
            if item_md.startswith("**Conrad:**") or item_md.startswith("**Sistema:**"):
                 display(Markdown("---"))
        # O loop continuará e o input() será chamado.